In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry
import plotly.express as px
from collections import namedtuple

In [ ]:
# Improting India 

In [2]:
url1 = 'https://api.covid19india.org/csv/latest/state_wise_daily.csv'
st_ws_daily = pd.read_csv(url1)

st_ws_daily.to_csv('st_ws_daily')
df = pd.melt(st_ws_daily,id_vars=['Date','Status'],var_name='Province/State',value_name='Cases')
df['Date']=pd.to_datetime(df['Date'])
df01=df.loc[df['Province/State']!='TT']
df1=df01.loc[df['Province/State']!='DN']
df1.head()

,Date,Status,Province/State,Cases
438,2020-03-14,Confirmed,AN,0
439,2020-03-14,Recovered,AN,0
440,2020-03-14,Deceased,AN,0
441,2020-03-15,Confirmed,AN,0
442,2020-03-15,Recovered,AN,0


In [3]:
ck=df1.loc[df1['Province/State']=='TT']
ck.shape

(0, 4)

In [4]:
st_ws_daily.head()

,Date,Status,TT,AN,AP,AR,AS,BR,CH,CT,...,PB,RJ,SK,TN,TG,TR,UP,UT,WB,UN
0,14-Mar-20,Confirmed,81,0,1,0,0,0,0,0,...,1,3,0,1,1,0,12,0,0,0
1,14-Mar-20,Recovered,9,0,0,0,0,0,0,0,...,0,1,0,0,0,0,4,0,0,0
2,14-Mar-20,Deceased,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15-Mar-20,Confirmed,27,0,0,0,0,0,0,0,...,0,1,0,0,2,0,1,0,0,0
4,15-Mar-20,Recovered,4,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,0


In [5]:
st_ws_daily.dtypes

Date      object
Status    object
TT         int64
AN         int64
AP         int64
AR         int64
AS         int64
BR         int64
CH         int64
CT         int64
DN         int64
DD         int64
DL         int64
GA         int64
GJ         int64
HR         int64
HP         int64
JK         int64
JH         int64
KA         int64
KL         int64
LA         int64
LD         int64
MP         int64
MH         int64
MN         int64
ML         int64
MZ         int64
NL         int64
OR         int64
PY         int64
PB         int64
RJ         int64
SK         int64
TN         int64
TG         int64
TR         int64
UP         int64
UT         int64
WB         int64
UN         int64
dtype: object

In [6]:
states=df1['Province/State'].unique()
states

array(['AN', 'AP', 'AR', 'AS', 'BR', 'CH', 'CT', 'DD', 'DL', 'GA', 'GJ',
       'HR', 'HP', 'JK', 'JH', 'KA', 'KL', 'LA', 'LD', 'MP', 'MH', 'MN',
       'ML', 'MZ', 'NL', 'OR', 'PY', 'PB', 'RJ', 'SK', 'TN', 'TG', 'TR',
       'UP', 'UT', 'WB', 'UN'], dtype=object)

In [7]:
st_ws_daily.shape

(438, 41)

In [8]:
df1.isnull().sum()

Date              0
Status            0
Province/State    0
Cases             0
dtype: int64

In [9]:
df1.nunique()

Date               146
Status               3
Province/State      37
Cases             1578
dtype: int64

In [10]:
print(df1['Date'].min(),df1['Date'].max())

2020-03-14 00:00:00 2020-08-06 00:00:00


In [11]:
df1['SNo']='1'
df1['Country']='India'

Confirmed_df=df1.loc[df1['Status']=='Confirmed']
Recovered_df=df1.loc[df1['Status']=='Recovered']
Deceased_df=df1.loc[df1['Status']=='Deceased']
all_countries = df1['Country'].unique()

Confirmed_df = Confirmed_df.drop(columns = ['Status'])
Recovered_df = Recovered_df.drop(columns = ['Status'])
Deceased_df = Deceased_df.drop(columns = ['Status'])

confirmed_df = Confirmed_df.rename(columns={'Cases': 'Confirmed'})
deaths_df = Deceased_df.rename(columns={'Cases': 'Deaths'})
recovered_df = Recovered_df.rename(columns={'Cases': 'Recovered'})

confirmed_df = confirmed_df[['SNo', 'Date','Province/State', 'Country', 'Confirmed']]
deaths_df = deaths_df[['SNo', 'Date','Province/State', 'Country', 'Deaths']]
recovered_df = recovered_df[['SNo', 'Date','Province/State', 'Country', 'Recovered']]

/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
print(confirmed_df.shape)
print(deaths_df.shape)
print(recovered_df.shape)

(5402, 5)
(5402, 5)
(5402, 5)


In [15]:
days_to_forecast = 22 # changable
first_forecasted_date = df1['Date'].max()


print('The first date to perform forecasts for is: ' + str(first_forecasted_date))

The first date to perform forecasts for is: 2020-08-06 00:00:00


In [16]:
first_forecasted_date = df1['Date'].max()
first_forecasted_date 

Timestamp('2020-08-06 00:00:00')

In [ ]:
# Covid Cases Prediction using Prophet 

In [17]:
def make_forecasts(all_countries, confirmed_df, deaths_df, recovered_df, days_to_forecast, first_forecasted_date, mode):
    forecast_dfs = []
    absolute_errors = [] # collate absolute errors so that we can find MAE later on
    counter = 0 # arbitrary counter to output snippet of result_val_df only once, when it is 1
    for country in all_countries:
        try:
            assert(country in confirmed_df['Country'].values)
            print('Country ' + str(country) + ' is listed! ')
            country_confirmed_df = confirmed_df[(confirmed_df['Country'] == country)]
            country_deaths_df = deaths_df[(deaths_df['Country'] == country)]
            country_recovered_df = recovered_df[(recovered_df['Country'] == country)]
            country_dfs = [('Confirmed', country_confirmed_df), 
                           ('Deaths', country_deaths_df), 
                           ('Recovered', country_recovered_df)]
            states_in_country = country_confirmed_df['Province/State'].unique()
            for state in states_in_country:
                try:
                    state_dfs = [] # to store forecasts for Confirmed, Deaths and Recovered

                    assert(state in country_confirmed_df['Province/State'].values)

                    # make forecasts for each case type (Confirmed, Deaths, Recovered)
                    for country_df_tup in country_dfs:
                        case_type = country_df_tup[0]
                        country_df = country_df_tup[1]
                        state_df = country_df[(country_df['Province/State'] == state)]

                        # data preparation for forecast with Prophet at state level
                        state_df = state_df[['Date', case_type]]
                        state_df.columns = ['ds','y']
                        state_df['ds'] = pd.to_datetime(state_df['ds'])

                        state_df_val = state_df[(state_df['ds'] >= pd.to_datetime(first_forecasted_date))] # validation set
                        state_df = state_df[(state_df['ds'] < pd.to_datetime(first_forecasted_date))] # train set

                        if mode == 'default':
                            m = Prophet()
                        elif mode == 'custom':
                            m = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
                            m.add_seasonality(name='monthly', period=30.5, fourier_order=10)
                            m.add_seasonality(name='weekly', period=7, fourier_order=21)
                            m.add_seasonality(name='daily', period=1, fourier_order=3)
                        m.fit(state_df)
                        future = m.make_future_dataframe(periods=days_to_forecast)
                        forecast = m.predict(future)

                        # evaluate forecasts with validation set and save absolute errors to absolute_errors
                        forecast_df = forecast[['ds', 'yhat']]
                        result_df = forecast_df[(forecast_df['ds'] >= pd.to_datetime(first_forecasted_date))]
                        result_val_df = result_df.merge(state_df_val, on=['ds'])
                        result_val_df['abs_diff'] = (result_val_df['y'] - result_val_df['yhat']).abs()
                        absolute_errors += list(result_val_df['abs_diff'].values)

                        # the following allows the user to check the output at particular checkpoints. Feel free to change!
                        if counter == 1:
                            print('Printing snippet of result_val_df: \n')
                            print(result_val_df)
                            print(absolute_errors)
                        counter += 1

                        # save results to dataframe
                        forecast_df['Province/State'] = state
                        forecast_df['Country/Region'] = country
                        forecast_df.rename(columns={'yhat':case_type}, inplace=True)
                        state_dfs += [forecast_df.tail(days_to_forecast)]

                    merged_df = state_dfs[0].merge(state_dfs[1],on=['ds', 'Province/State', 'Country/Region']).merge(state_dfs[2],on=['ds', 'Province/State', 'Country/Region'])
                    forecast_dfs += [merged_df]
                except:
                    continue
        except:
            print('Country ' + str(country) + ' is not listed! ')
            continue
    results_tup = namedtuple('results_tup', ['forecast_dfs', 'absolute_errors'])
    return results_tup(forecast_dfs, absolute_errors)

In [18]:
forecast_results = make_forecasts(all_countries, confirmed_df, deaths_df, recovered_df, days_to_forecast, first_forecasted_date, 'default')
absolute_errors = forecast_results.absolute_errors

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Country India is listed! 


/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

Printing snippet of result_val_df: 

          ds      yhat  y  abs_diff
0 2020-08-06  0.549124  2  1.450876
[49.339265262739055, 1.4508762207739139]


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [19]:
forecasts_final = pd.concat(forecast_results.forecast_dfs, axis=0)
forecasts_final.sort_values(by='ds')
forecasts_final = forecasts_final[['ds', 'Province/State', 'Country/Region', 'Deaths', 'Recovered']]
forecasts_final.rename(columns={'ds':'ObservationDate'}, inplace=True)
for case_type in ['Deaths', 'Recovered']:
    forecasts_final[case_type] = forecasts_final[case_type].round() # round forecasts to integer as humans cannot be floats
    #forecasts_final[forecasts_final[case_type] < 0] = 0 # replace negative forecasts to zero

forecasts_final

,ObservationDate,Province/State,Country/Region,Deaths,Recovered
0,2020-08-06,AN,India,1.0,10.0
1,2020-08-07,AN,India,1.0,10.0
2,2020-08-08,AN,India,1.0,11.0
3,2020-08-09,AN,India,1.0,11.0
4,2020-08-10,AN,India,1.0,11.0
...,...,...,...,...,...
17,2020-08-23,UN,India,0.0,0.0
18,2020-08-24,UN,India,0.0,0.0
19,2020-08-25,UN,India,0.0,0.0
20,2020-08-26,UN,India,0.0,0.0


In [20]:
forecasts_final['PREDICTION'] = forecasts_final['Deaths']+forecasts_final['Recovered']
forecasts_final = forecasts_final.rename(columns={'ObservationDate':'DATE','Province/State': 'STATE_CD'})
forecasts_final = forecasts_final [['DATE','STATE_CD','PREDICTION']]
forecasts_final

,DATE,STATE_CD,PREDICTION
0,2020-08-06,AN,11.0
1,2020-08-07,AN,11.0
2,2020-08-08,AN,12.0
3,2020-08-09,AN,12.0
4,2020-08-10,AN,12.0
...,...,...,...
17,2020-08-23,UN,0.0
18,2020-08-24,UN,0.0
19,2020-08-25,UN,0.0
20,2020-08-26,UN,0.0


In [21]:
forecasts_final['DATE'].min(),forecasts_final['DATE'].max()

(Timestamp('2020-08-06 00:00:00'), Timestamp('2020-08-27 00:00:00'))

In [22]:
forecasts_final_india = forecasts_final.loc[forecasts_final['DATE']>'2020-08-06']

In [23]:
forecasts_final_india['DATE'].min(),forecasts_final_india['DATE'].max()

(Timestamp('2020-08-07 00:00:00'), Timestamp('2020-08-27 00:00:00'))

In [24]:
forecasts_final_india.shape

(777, 3)

In [25]:
forecasts_final_india.to_csv('FinalSubmission.csv',index=False)

In [26]:
forecasts_final_india['STATE_CD'].value_counts()

SK    21
AR    21
AS    21
DL    21
GJ    21
UT    21
RJ    21
AP    21
TR    21
BR    21
TN    21
UP    21
KL    21
HR    21
CT    21
OR    21
UN    21
JK    21
MN    21
CH    21
NL    21
ML    21
MZ    21
DD    21
PB    21
HP    21
MP    21
LD    21
MH    21
AN    21
KA    21
WB    21
LA    21
TG    21
PY    21
GA    21
JH    21
Name: STATE_CD, dtype: int64

In [27]:
DL=forecasts_final.loc[forecasts_final['STATE_CD']=='DL']
DL

,DATE,STATE_CD,PREDICTION
0,2020-08-06,DL,2454.0
1,2020-08-07,DL,2252.0
2,2020-08-08,DL,2613.0
3,2020-08-09,DL,2309.0
4,2020-08-10,DL,2207.0
5,2020-08-11,DL,2172.0
6,2020-08-12,DL,2263.0
7,2020-08-13,DL,2579.0
8,2020-08-14,DL,2377.0
9,2020-08-15,DL,2739.0


In [28]:
MH=forecasts_final.loc[forecasts_final['STATE_CD']=='MH']
MH

,DATE,STATE_CD,PREDICTION
0,2020-08-06,MH,8670.0
1,2020-08-07,MH,8628.0
2,2020-08-08,MH,8797.0
3,2020-08-09,MH,8595.0
4,2020-08-10,MH,9089.0
5,2020-08-11,MH,9364.0
6,2020-08-12,MH,8939.0
7,2020-08-13,MH,9574.0
8,2020-08-14,MH,9532.0
9,2020-08-15,MH,9702.0


In [29]:
KA=forecasts_final.loc[forecasts_final['STATE_CD']=='KA']
KA

,DATE,STATE_CD,PREDICTION
0,2020-08-06,KA,3999.0
1,2020-08-07,KA,4055.0
2,2020-08-08,KA,4191.0
3,2020-08-09,KA,4284.0
4,2020-08-10,KA,4355.0
5,2020-08-11,KA,4644.0
6,2020-08-12,KA,4646.0
7,2020-08-13,KA,4762.0
8,2020-08-14,KA,4817.0
9,2020-08-15,KA,4953.0


In [159]:
TN=forecasts_final.loc[forecasts_final['STATE_CD']=='TN']
TN

,DATE,STATE_CD,PREDICTION
0,2020-07-27,TN,5496.0
1,2020-07-28,TN,5842.0
2,2020-07-29,TN,5961.0
3,2020-07-30,TN,6091.0
4,2020-07-31,TN,6161.0
5,2020-08-01,TN,6288.0
6,2020-08-02,TN,6244.0
7,2020-08-03,TN,6306.0
8,2020-08-04,TN,6653.0
9,2020-08-05,TN,6772.0


In [30]:
RJ=forecasts_final.loc[forecasts_final['STATE_CD']=='RJ']
RJ

,DATE,STATE_CD,PREDICTION
0,2020-08-06,RJ,737.0
1,2020-08-07,RJ,777.0
2,2020-08-08,RJ,786.0
3,2020-08-09,RJ,751.0
4,2020-08-10,RJ,803.0
5,2020-08-11,RJ,781.0
6,2020-08-12,RJ,849.0
7,2020-08-13,RJ,813.0
8,2020-08-14,RJ,853.0
9,2020-08-15,RJ,862.0


In [88]:
df1.head()

,Date,Status,Province/State,Cases,SNo,Country
405,2020-03-14,Confirmed,AN,0,1,India
406,2020-03-14,Recovered,AN,0,1,India
407,2020-03-14,Deceased,AN,0,1,India
408,2020-03-15,Confirmed,AN,0,1,India
409,2020-03-15,Recovered,AN,0,1,India


In [ ]:
# For US

In [29]:
us=pd.read_csv('covid_confirmed_usafacts.csv')
us_df = pd.melt(us,id_vars=['countyFIPS','County Name','State','stateFIPS'],var_name='Date',value_name='Cases')
us_df['Date']=pd.to_datetime(us_df['Date'])
us_df1=us_df.groupby(['Date','State']).Cases.sum().reset_index()
us_df1['SNo']='1'
us_df1['Country']='US'
confirmed_df_us = us_df1.rename(columns={'Cases': 'Confirmed','State':'Province/State'})

In [30]:
us_df.nunique()

countyFIPS      3146
County Name     1882
State             51
stateFIPS         51
Date             188
Cases          11463
dtype: int64

In [31]:
print(confirmed_df_us['Date'].min(),confirmed_df_us['Date'].max())

2020-01-22 00:00:00 2020-07-27 00:00:00


In [32]:
confirmed_df_us.isnull().sum()

Date              0
Province/State    0
Confirmed         0
SNo               0
Country           0
dtype: int64

In [54]:
days_to_forecast = 24 # changable
first_forecasted_date = confirmed_df_us['Date'].max()

In [34]:
#confirmed_df2=confirmed_df.loc[confirmed_df['Date']<'2020-07-20']
confirmed_df2=confirmed_df_us

In [35]:
states=confirmed_df_us['Province/State'].unique()
states

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [36]:
all_countries = confirmed_df_us['Country'].unique()
all_countries

array(['US'], dtype=object)

In [37]:
confirmed_df_us.head()

,Date,Province/State,Confirmed,SNo,Country
0,2020-01-22,AK,0,1,US
1,2020-01-22,AL,0,1,US
2,2020-01-22,AR,0,1,US
3,2020-01-22,AZ,0,1,US
4,2020-01-22,CA,0,1,US


In [38]:
confirmed_df_us.shape

(9588, 5)

In [55]:
def make_forecasts(all_countries, confirmed_df,  days_to_forecast, first_forecasted_date, mode):
    forecast_dfs = []
    absolute_errors = [] # collate absolute errors so that we can find MAE later on
    counter = 0 # arbitrary counter to output snippet of result_val_df only once, when it is 1
    for country in all_countries:
        try:
            assert(country in confirmed_df['Country'].values)
            print('Country ' + str(country) + ' is listed! ')
            country_confirmed_df = confirmed_df[(confirmed_df['Country'] == country)]
            #country_deaths_df = deaths_df[(deaths_df['Country'] == country)]
            #country_recovered_df = recovered_df[(recovered_df['Country'] == country)]
            country_dfs = [('Confirmed', country_confirmed_df)]
            states_in_country = country_confirmed_df['Province/State'].unique()
            for state in states_in_country:
                try:
                    state_dfs = [] # to store forecasts for Confirmed, Deaths and Recovered

                    assert(state in country_confirmed_df['Province/State'].values)

                    # make forecasts for each case type (Confirmed, Deaths, Recovered)
                    for country_df_tup in country_dfs:
                        case_type = country_df_tup[0]
                        country_df = country_df_tup[1]
                        state_df = country_df[(country_df['Province/State'] == state)]

                        # data preparation for forecast with Prophet at state level
                        state_df = state_df[['Date', case_type]]
                        state_df.columns = ['ds','y']
                        state_df['ds'] = pd.to_datetime(state_df['ds'])

                        state_df_val = state_df[(state_df['ds'] >= pd.to_datetime(first_forecasted_date))] # validation set
                        state_df = state_df[(state_df['ds'] < pd.to_datetime(first_forecasted_date))] # train set

                        if mode == 'default':
                            m = Prophet()
                        elif mode == 'custom':
                            m = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
                            m.add_seasonality(name='monthly', period=30.5, fourier_order=10)
                            m.add_seasonality(name='weekly', period=7, fourier_order=21)
                            m.add_seasonality(name='daily', period=1, fourier_order=3)
                        m.fit(state_df)
                        future = m.make_future_dataframe(periods=days_to_forecast)
                        forecast = m.predict(future)

                        # evaluate forecasts with validation set and save absolute errors to absolute_errors
                        forecast_df = forecast[['ds', 'yhat']]
                        result_df = forecast_df[(forecast_df['ds'] >= pd.to_datetime(first_forecasted_date))]
                        result_val_df = result_df.merge(state_df_val, on=['ds'])
                        result_val_df['abs_diff'] = (result_val_df['y'] - result_val_df['yhat']).abs()
                        absolute_errors += list(result_val_df['abs_diff'].values)

                        # the following allows the user to check the output at particular checkpoints. Feel free to change!
                        if counter == 1:
                            print('Printing snippet of result_val_df: \n')
                            print(result_val_df)
                            print(absolute_errors)
                        counter += 1

                        # save results to dataframe
                        forecast_df['Province/State'] = state
                        forecast_df['Country/Region'] = country
                        forecast_df.rename(columns={'yhat':case_type}, inplace=True)
                        state_dfs += [forecast_df.tail(days_to_forecast)]

                    #merged_df = state_dfs[0].merge(state_dfs[1],on=['ds', 'Province/State', 'Country/Region']).merge(state_dfs[2],on=['ds', 'Province/State', 'Country/Region'])
                    merged_df = state_dfs[0]
                    forecast_dfs += [merged_df]
                except:
                    continue
        except:
            print('Country ' + str(country) + ' is not listed! ')
            continue
    results_tup = namedtuple('results_tup', ['forecast_dfs', 'absolute_errors'])
    return results_tup(forecast_dfs, absolute_errors)

In [56]:
forecast_results_us = make_forecasts(all_countries, confirmed_df_us, days_to_forecast, first_forecasted_date, 'default')
absolute_errors = forecast_results_us.absolute_errors

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Country US is listed! 


/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonal

Printing snippet of result_val_df: 

          ds          yhat      y     abs_diff
0 2020-07-27  75739.976207  82295  6555.023793
[470.19386127650523, 6555.023793289583]


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [57]:
forecasts_final_us = pd.concat(forecast_results_us.forecast_dfs, axis=0)
forecasts_final_us.sort_values(by='ds')
forecasts_final_us = forecasts_final_us[['ds', 'Province/State', 'Country/Region', 'Confirmed']]
forecasts_final_us.rename(columns={'ds':'ObservationDate'}, inplace=True)
for case_type in ['Confirmed']:
    forecasts_final_us[case_type] = forecasts_final_us[case_type].round() # round forecasts to integer as humans cannot be floats
    #forecasts_final_us[forecasts_final[case_type] < 0] = 0 # replace negative forecasts to zero

forecasts_final_us

,ObservationDate,Province/State,Country/Region,Confirmed
187,2020-07-27,AK,US,2153.0
188,2020-07-28,AK,US,2193.0
189,2020-07-29,AK,US,2236.0
190,2020-07-30,AK,US,2276.0
191,2020-07-31,AK,US,2319.0
...,...,...,...,...
206,2020-08-15,WY,US,3071.0
207,2020-08-16,WY,US,3101.0
208,2020-08-17,WY,US,3132.0
209,2020-08-18,WY,US,3172.0


In [58]:
forecasts_final_us = forecasts_final_us.rename(columns={'ObservationDate':'DATE','Province/State': 'STATE_CD','Confirmed':'PREDICTION'})
forecasts_final_us= forecasts_final_us[['DATE','STATE_CD','PREDICTION']]
forecasts_final_us

,DATE,STATE_CD,PREDICTION
187,2020-07-27,AK,2153.0
188,2020-07-28,AK,2193.0
189,2020-07-29,AK,2236.0
190,2020-07-30,AK,2276.0
191,2020-07-31,AK,2319.0
...,...,...,...
206,2020-08-15,WY,3071.0
207,2020-08-16,WY,3101.0
208,2020-08-17,WY,3132.0
209,2020-08-18,WY,3172.0


In [59]:
forecasts_final_us = forecasts_final_us.loc[forecasts_final_us['DATE']>'2020-07-30']

In [60]:
forecasts_final_us['DATE'].min(),forecasts_final_us['DATE'].max()

(Timestamp('2020-07-31 00:00:00'), Timestamp('2020-08-19 00:00:00'))

In [62]:
forecasts_final_us.shape

(1020, 3)

In [63]:
forecasts_final_us['STATE_CD'].value_counts()

ME    20
AR    20
RI    20
GA    20
IA    20
CA    20
TX    20
DC    20
MT    20
NJ    20
HI    20
LA    20
AZ    20
KS    20
OR    20
NM    20
MA    20
MS    20
OK    20
TN    20
WY    20
NC    20
AL    20
ND    20
KY    20
WA    20
FL    20
IL    20
WI    20
UT    20
MI    20
VT    20
NE    20
AK    20
ID    20
VA    20
NH    20
SC    20
NY    20
MD    20
NV    20
SD    20
CO    20
OH    20
IN    20
MN    20
CT    20
DE    20
MO    20
PA    20
WV    20
Name: STATE_CD, dtype: int64

In [ ]:
# Concate India and US final Predication

In [64]:
FinalSubmission=forecasts_final_india.append(forecasts_final_us,ignore_index=True)

In [67]:
print(forecasts_final_india.shape)
print(forecasts_final_us.shape)

(740, 3)
(1020, 3)


In [65]:
FinalSubmission.head()

,DATE,STATE_CD,PREDICTION
0,2020-07-31,AN,4.0
1,2020-08-01,AN,4.0
2,2020-08-02,AN,4.0
3,2020-08-03,AN,5.0
4,2020-08-04,AN,5.0


In [68]:
FinalSubmission.to_csv('FinalSubmission.csv',index=False)

In [66]:
FinalSubmission.shape

(1760, 3)

In [50]:
NY=forecasts_final_us.loc[forecasts_final_us['STATE_CD']=='NY']
NY

,DATE,STATE_CD,PREDICTION
187,2020-07-27,NY,412005.0
188,2020-07-28,NY,412223.0
189,2020-07-29,NY,413110.0
190,2020-07-30,NY,414037.0
191,2020-07-31,NY,415059.0
192,2020-08-01,NY,416131.0
193,2020-08-02,NY,416513.0
194,2020-08-03,NY,416733.0
195,2020-08-04,NY,416951.0
196,2020-08-05,NY,417837.0


In [51]:
NJ=forecasts_final_us.loc[forecasts_final_us['STATE_CD']=='NJ']
NJ

,DATE,STATE_CD,PREDICTION
187,2020-07-27,NJ,179146.0
188,2020-07-28,NJ,179414.0
189,2020-07-29,NJ,179612.0
190,2020-07-30,NJ,180048.0
191,2020-07-31,NJ,180381.0
192,2020-08-01,NJ,180679.0
193,2020-08-02,NJ,180917.0
194,2020-08-03,NJ,181113.0
195,2020-08-04,NJ,181381.0
196,2020-08-05,NJ,181579.0


In [52]:
PA=forecasts_final_us.loc[forecasts_final_us['STATE_CD']=='PA']
PA

,DATE,STATE_CD,PREDICTION
187,2020-07-27,PA,106227.0
188,2020-07-28,PA,106916.0
189,2020-07-29,PA,107618.0
190,2020-07-30,PA,108400.0
191,2020-07-31,PA,109205.0
192,2020-08-01,PA,109959.0
193,2020-08-02,PA,110602.0
194,2020-08-03,PA,111115.0
195,2020-08-04,PA,111803.0
196,2020-08-05,PA,112505.0


In [53]:
forecasts_final_us['DATE'].min(),forecasts_final_us['DATE'].max()

(Timestamp('2020-07-27 00:00:00'), Timestamp('2020-08-18 00:00:00'))

In [52]:
forecasts_final['DATE'].min(),forecasts_final['DATE'].max()

(Timestamp('2020-07-20 00:00:00'), Timestamp('2020-07-26 00:00:00'))